# 🏈 Football Analytics Platform - Démonstration

Ce notebook présente les fonctionnalités principales de la plateforme d'intelligence football analytics.

## 📋 Sommaire
1. [Configuration et Données](#config)
2. [Analyse de Performance Joueurs](#performance)
3. [Métriques Football Avancées](#metrics)
4. [Analyses Tactiques](#tactics)
5. [Scouting et Recrutement](#scouting)
6. [Visualisations](#viz)

---

## 1. Configuration et Chargement des Données {#config}

In [ ]:
# Imports et configuration
import sys
import os
import warnings
warnings.filterwarnings('ignore')

# Ajout des chemins pour nos modules
sys.path.append('../python_analytics/modules')
sys.path.append('../configs')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Nos modules personnalisés
from performance_analyzer import PlayerPerformanceAnalyzer, TacticalAnalyzer, FootballMetrics
from scouting_engine import ScoutingEngine, PlayerProfiler, MarketValuePredictor
from database import DatabaseManager

# Configuration graphiques
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
%matplotlib inline

print("🚀 Football Analytics Platform")
print("==============================")
print("✅ Modules chargés avec succès!")

In [ ]:
# Connexion à la base de données
db = DatabaseManager()

if db.connect():
    print("✅ Connexion DB établie")
    
    # Test de requête
    teams = db.read_sql("SELECT name, city, budget_millions FROM teams LIMIT 5")
    print("\n📊 Échantillon d'équipes:")
    display(teams)
else:
    print("⚠️ Utilisation de données simulées")
    teams = pd.DataFrame({
        'name': ['PSG', 'OM', 'Lyon', 'Monaco'],
        'city': ['Paris', 'Marseille', 'Lyon', 'Monaco'],
        'budget_millions': [600, 150, 180, 200]
    })
    display(teams)

## 2. Analyse de Performance Joueurs {#performance}

In [ ]:
# Initialisation de l'analyseur de performance
performance_analyzer = PlayerPerformanceAnalyzer(db.connection if db.connection else None)

# Analyse de forme d'un joueur fictif
player_id = "player_1"
form_analysis = performance_analyzer.get_player_form(player_id, last_n_matches=10)

print("📊 ANALYSE DE FORME JOUEUR")
print("="*30)

if "error" not in form_analysis:
    print(f"🎯 Note moyenne: {form_analysis['avg_rating']:.2f}/10")
    print(f"⚽ Goals/90min: {form_analysis['goals_per_90']:.2f}")
    print(f"🅰️ Assists/90min: {form_analysis['assists_per_90']:.2f}")
    print(f"📈 Tendance: {form_analysis['rating_trend']}")
    print(f"🎯 Consistance: {form_analysis['consistency_score']:.1f}/100")
else:
    print("⚠️ Simulation de données de performance")
    # Générer des données de démo pour l'affichage
    form_analysis = {
        'avg_rating': 7.8,
        'goals_per_90': 0.72,
        'assists_per_90': 0.45,
        'rating_trend': 'En hausse',
        'consistency_score': 82.5
    }
    print(f"🎯 Note moyenne: {form_analysis['avg_rating']:.2f}/10")
    print(f"⚽ Goals/90min: {form_analysis['goals_per_90']:.2f}")
    print(f"🅰️ Assists/90min: {form_analysis['assists_per_90']:.2f}")
    print(f"📈 Tendance: {form_analysis['rating_trend']}")
    print(f"🎯 Consistance: {form_analysis['consistency_score']:.1f}/100")

In [ ]:
# Visualisation de l'évolution des performances
np.random.seed(42)
dates = pd.date_range('2024-08-01', periods=10, freq='W')
ratings = np.random.uniform(6.5, 9.0, 10)
goals = np.random.poisson(0.5, 10)
assists = np.random.poisson(0.3, 10)

fig = make_subplots(
    rows=2, cols=1,
    subplot_titles=("Évolution des Notes", "Contributions Offensives"),
    shared_xaxes=True
)

# Graphique des notes
fig.add_trace(
    go.Scatter(x=dates, y=ratings, mode='lines+markers', 
               name='Note/10', line=dict(color='blue', width=3)),
    row=1, col=1
)

# Ligne de moyenne
fig.add_hline(y=ratings.mean(), line_dash="dash", 
              annotation_text=f"Moyenne: {ratings.mean():.1f}",
              row=1, col=1)

# Graphique des contributions
fig.add_trace(
    go.Bar(x=dates, y=goals, name='Goals', marker_color='red'),
    row=2, col=1
)
fig.add_trace(
    go.Bar(x=dates, y=assists, name='Assists', marker_color='orange'),
    row=2, col=1
)

fig.update_layout(
    title="📈 Performance Joueur - 10 Derniers Matchs",
    height=600,
    showlegend=True
)

fig.show()

## 3. Métriques Football Avancées {#metrics}

In [ ]:
# Calcul des métriques xG (Expected Goals)
metrics_calculator = FootballMetrics()

# Simulation de données de tirs
np.random.seed(42)
shots_data = pd.DataFrame({
    'x_coordinate': np.random.uniform(60, 100, 50),  # Zone offensive
    'y_coordinate': np.random.uniform(20, 80, 50),   # Largeur terrain
    'event_details': [{'situation': np.random.choice(['open_play', 'counter_attack', 'set_piece'])} for _ in range(50)]
})

# Calcul xG
shots_with_xg = metrics_calculator.calculate_xg(shots_data)

print("⚽ MÉTRIQUES EXPECTED GOALS (xG)")
print("="*35)
print(f"🎯 Nombre de tirs: {len(shots_with_xg)}")
print(f"📊 xG total: {shots_with_xg['xg'].sum():.2f}")
print(f"📈 xG moyen par tir: {shots_with_xg['xg'].mean():.3f}")
print(f"🏆 Meilleur xG: {shots_with_xg['xg'].max():.3f}")

# Analyse par situation
situation_xg = shots_with_xg.groupby(
    shots_with_xg['event_details'].apply(lambda x: x.get('situation', 'open_play') if isinstance(x, dict) else 'open_play')
)['xg'].agg(['count', 'sum', 'mean']).round(3)

print("\n📋 xG par situation de jeu:")
display(situation_xg)

In [ ]:
# Visualisation des zones de tir et xG
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Carte des tirs
scatter = ax1.scatter(shots_with_xg['x_coordinate'], shots_with_xg['y_coordinate'], 
                     c=shots_with_xg['xg'], s=shots_with_xg['xg']*200, 
                     cmap='Reds', alpha=0.7)

ax1.set_xlim(50, 100)
ax1.set_ylim(0, 100)
ax1.set_xlabel('Position X (50=Milieu, 100=But)')
ax1.set_ylabel('Position Y (0-100 Largeur)')
ax1.set_title('🎯 Carte des Tirs avec xG\n(Taille = Valeur xG)')
ax1.grid(True, alpha=0.3)

# Ligne de but
ax1.axvline(x=100, color='black', linewidth=3, label='Ligne de but')
ax1.axhline(y=44, color='black', linewidth=2, alpha=0.5)
ax1.axhline(y=56, color='black', linewidth=2, alpha=0.5)

plt.colorbar(scatter, ax=ax1, label='Expected Goals (xG)')

# Distribution xG
ax2.hist(shots_with_xg['xg'], bins=20, alpha=0.7, color='skyblue', edgecolor='black')
ax2.axvline(shots_with_xg['xg'].mean(), color='red', linestyle='--', 
           label=f'Moyenne: {shots_with_xg["xg"].mean():.3f}')
ax2.set_xlabel('Expected Goals (xG)')
ax2.set_ylabel('Nombre de Tirs')
ax2.set_title('📊 Distribution des Valeurs xG')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 4. Analyses Tactiques {#tactics}

In [ ]:
# Analyse tactique d'équipe
tactical_analyzer = TacticalAnalyzer(db.connection if db.connection else None)

# Simulation d'une analyse tactique
print("⚔️ ANALYSE TACTIQUE D'ÉQUIPE")
print("="*30)

# Données tactiques simulées
tactical_data = {
    'formation_detected': '4-3-3',
    'possession_avg': 58.3,
    'ppda': 11.4,
    'passes_per_match': 542,
    'pass_accuracy': 87.1,
    'pressing_intensity': 'Intense'
}

print(f"🛡️ Formation détectée: {tactical_data['formation_detected']}")
print(f"⚽ Possession moyenne: {tactical_data['possession_avg']:.1f}%")
print(f"🔥 PPDA (Pressing): {tactical_data['ppda']:.1f} - {tactical_data['pressing_intensity']}")
print(f"📊 Passes/match: {tactical_data['passes_per_match']}")
print(f"🎯 Précision passes: {tactical_data['pass_accuracy']:.1f}%")

In [ ]:
# Visualisation des positions moyennes (formation 4-3-3)
fig, ax = plt.subplots(figsize=(12, 8))

# Positions des joueurs (formation 4-3-3)
positions = {
    'GK': (10, 50),
    'LB': (25, 20), 'CB1': (25, 40), 'CB2': (25, 60), 'RB': (25, 80),
    'DM': (45, 50), 'CM1': (55, 35), 'CM2': (55, 65),
    'LW': (75, 25), 'ST': (80, 50), 'RW': (75, 75)
}

# Dessiner le terrain
ax.add_patch(plt.Rectangle((0, 0), 100, 100, fill=False, edgecolor='white', linewidth=3))
ax.axvline(x=50, color='white', linewidth=2)
ax.add_patch(plt.Circle((50, 50), 9.15, fill=False, edgecolor='white', linewidth=2))

# Surface de réparation
ax.add_patch(plt.Rectangle((0, 21.1), 16.5, 57.8, fill=False, edgecolor='white', linewidth=2))
ax.add_patch(plt.Rectangle((83.5, 21.1), 16.5, 57.8, fill=False, edgecolor='white', linewidth=2))

# Placer les joueurs
for pos, (x, y) in positions.items():
    color = 'red' if pos == 'GK' else 'blue' if 'B' in pos else 'green' if 'M' in pos or pos == 'DM' else 'orange'
    ax.scatter(x, y, s=500, c=color, alpha=0.8, edgecolors='black', linewidth=2)
    ax.annotate(pos, (x, y), ha='center', va='center', fontweight='bold', fontsize=10)

# Zones d'influence (cercles)
for pos, (x, y) in positions.items():
    if pos != 'GK':
        ax.add_patch(plt.Circle((x, y), 8, fill=False, edgecolor='gray', alpha=0.3, linestyle='--'))

ax.set_xlim(0, 100)
ax.set_ylim(0, 100)
ax.set_facecolor('#2d5a2d')  # Couleur terrain
ax.set_title('⚽ Formation 4-3-3 - Positions Moyennes\nPSG vs OM', 
             fontsize=16, fontweight='bold', color='white', pad=20)
ax.set_xlabel('Longueur du terrain (0=But défensif, 100=But offensif)', color='white')
ax.set_ylabel('Largeur du terrain', color='white')

# Légende
legend_elements = [
    plt.scatter([], [], c='red', s=100, label='Gardien'),
    plt.scatter([], [], c='blue', s=100, label='Défenseurs'),
    plt.scatter([], [], c='green', s=100, label='Milieux'),
    plt.scatter([], [], c='orange', s=100, label='Attaquants')
]
ax.legend(handles=legend_elements, loc='upper right', bbox_to_anchor=(1, 1))

plt.tight_layout()
plt.show()

## 5. Scouting et Recrutement {#scouting}

In [ ]:
# Moteur de scouting et recrutement
scouting_engine = ScoutingEngine()

# Chargement de la base de données joueurs (simulation)
print("🔍 MOTEUR DE SCOUTING & RECRUTEMENT")
print("="*40)

# Génération de données de démonstration
scouting_engine._load_demo_database()
print(f"📊 Base de données: {len(scouting_engine.player_database)} joueurs")

# Analyse des profils de joueurs
clusters = scouting_engine.profiler.player_clusters
print(f"🎯 Profils identifiés: {len(clusters)}")

for cluster_id, info in clusters.items():
    print(f"  • {info['label']}: {info['size']} joueurs")

In [ ]:
# Recherche de joueurs selon des critères
criteria = {
    'position': 'ST',
    'age_min': 20,
    'age_max': 28,
    'max_value': 50,
    'min_rating': 75
}

print(f"🎯 Recherche selon critères: {criteria}")
recommendations = scouting_engine.scout_by_criteria(criteria)

print(f"\n📋 {len(recommendations)} joueurs trouvés")

if recommendations:
    print("\n🔝 Top 5 recommandations:")
    for i, player in enumerate(recommendations[:5], 1):
        print(f"{i}. {player['name']} ({player['age']} ans) - "
              f"Profil: {player['cluster_label']} - "
              f"Score: {player['recommendation_score']:.1f}/100")

In [ ]:
# Analyse d'opportunité de transfert
player_id = recommendations[0]['player_id'] if recommendations else 'player_1'
budget = 30  # 30M€

opportunity = scouting_engine.analyze_transfer_opportunity(player_id, budget)

print("💰 ANALYSE D'OPPORTUNITÉ DE TRANSFERT")
print("="*40)

if "error" not in opportunity:
    player_info = opportunity['player_info']
    market_analysis = opportunity['market_analysis']
    
    print(f"👤 Joueur: {player_info['name']} ({player_info['age']} ans)")
    print(f"🎯 Position: {player_info['position']}")
    print(f"⚽ Équipe actuelle: {player_info['current_team']}")
    print(f"\n💰 Valeur estimée: {market_analysis['predicted_value']:.1f}M€")
    print(f"📊 Fourchette: {market_analysis['min_value']:.1f}-{market_analysis['max_value']:.1f}M€")
    print(f"🎯 Confiance: {market_analysis['confidence']}%")
    print(f"💳 Compatible budget: {'✅ Oui' if opportunity['budget_fit'] else '❌ Non'}")
    print(f"\n📈 Score d'opportunité: {opportunity['opportunity_score']:.1f}/100")
    print(f"🔮 Recommandation: {opportunity['recommendation']}")
    
    print("\n⚠️ Risques identifiés:")
    for risk in opportunity['risks']:
        print(f"  • {risk}")
    
    print("\n✅ Avantages:")
    for benefit in opportunity['benefits']:
        print(f"  • {benefit}")
else:
    print("⚠️ Simulation d'analyse de transfert")
    print("👤 Joueur: Kylian Mbappé (25 ans)")
    print("💰 Valeur estimée: 180M€")
    print("🔮 Recommandation: 🔴 Hors budget")

## 6. Visualisations Avancées {#viz}

In [ ]:
# Graphique radar de comparaison de joueurs
def create_player_radar(player_name, stats_dict):
    categories = list(stats_dict.keys())
    values = list(stats_dict.values())
    
    fig = go.Figure()
    
    fig.add_trace(go.Scatterpolar(
        r=values,
        theta=categories,
        fill='toself',
        name=player_name,
        line_color='blue'
    ))
    
    fig.update_layout(
        polar=dict(
            radialaxis=dict(
                visible=True,
                range=[0, 100]
            )
        ),
        title=f"🎯 Profil Radar - {player_name}",
        showlegend=True
    )
    
    return fig

# Statistiques exemple pour Mbappé
mbappe_stats = {
    'Finition': 92,
    'Vitesse': 96,
    'Dribble': 88,
    'Passes': 78,
    'Défense': 35,
    'Physique': 82,
    'Technique': 87,
    'Mental': 90
}

radar_fig = create_player_radar("Kylian Mbappé", mbappe_stats)
radar_fig.show()

In [ ]:
# Analyse comparative d'équipes
teams_comparison = pd.DataFrame({
    'Équipe': ['PSG', 'OM', 'Lyon', 'Monaco'],
    'Goals/Match': [2.3, 1.8, 2.0, 2.1],
    'xG/Match': [2.1, 1.6, 1.9, 2.0],
    'Possession %': [62, 48, 53, 55],
    'Passes/Match': [587, 421, 456, 478],
    'Précision Passes %': [87.1, 82.3, 84.7, 85.2],
    'PPDA': [11.4, 15.8, 13.2, 12.6]
})

# Heatmap de comparaison
plt.figure(figsize=(12, 6))

# Normaliser les données pour la heatmap
comparison_normalized = teams_comparison.set_index('Équipe')
for col in comparison_normalized.columns:
    if col != 'PPDA':  # Pour PPDA, plus bas = mieux
        comparison_normalized[col] = (comparison_normalized[col] - comparison_normalized[col].min()) / (comparison_normalized[col].max() - comparison_normalized[col].min()) * 100
    else:
        comparison_normalized[col] = (comparison_normalized[col].max() - comparison_normalized[col]) / (comparison_normalized[col].max() - comparison_normalized[col].min()) * 100

sns.heatmap(comparison_normalized.T, annot=True, fmt='.1f', cmap='RdYlGn', 
            cbar_kws={'label': 'Performance Relative (0-100)'}, 
            linewidths=0.5)

plt.title('🏆 Comparaison Multi-Critères des Équipes\n(Valeurs normalisées)', 
          fontsize=14, fontweight='bold', pad=20)
plt.xlabel('Équipes', fontweight='bold')
plt.ylabel('Métriques', fontweight='bold')
plt.xticks(rotation=0)
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()

print("📊 Données brutes de comparaison:")
display(teams_comparison)

In [ ]:
# Évolution saisonnière des performances d'équipe
np.random.seed(42)
matchdays = range(1, 16)
psg_points = np.cumsum(np.random.choice([0, 1, 3], 15, p=[0.1, 0.2, 0.7]))
om_points = np.cumsum(np.random.choice([0, 1, 3], 15, p=[0.2, 0.3, 0.5]))
lyon_points = np.cumsum(np.random.choice([0, 1, 3], 15, p=[0.15, 0.25, 0.6]))

fig = go.Figure()

fig.add_trace(go.Scatter(x=list(matchdays), y=psg_points, 
                        mode='lines+markers', name='PSG', 
                        line=dict(color='blue', width=3)))

fig.add_trace(go.Scatter(x=list(matchdays), y=om_points, 
                        mode='lines+markers', name='OM', 
                        line=dict(color='lightblue', width=3)))

fig.add_trace(go.Scatter(x=list(matchdays), y=lyon_points, 
                        mode='lines+markers', name='Lyon', 
                        line=dict(color='red', width=3)))

fig.update_layout(
    title="📈 Évolution des Points en Championnat",
    xaxis_title="Journée de Championnat",
    yaxis_title="Points Cumulés",
    hovermode='x unified',
    height=500
)

fig.show()

print(f"📊 Situation après 15 journées:")
print(f"1. PSG: {psg_points[-1]} points")
print(f"2. Lyon: {lyon_points[-1]} points")
print(f"3. OM: {om_points[-1]} points")

## 📋 Résumé et Conclusions

### ✅ Fonctionnalités Démontrées

1. **📊 Analyse de Performance**
   - Suivi de forme des joueurs
   - Métriques de consistance
   - Tendances et évolutions

2. **⚽ Métriques Avancées**
   - Calcul d'Expected Goals (xG)
   - Analyse spatiale des tirs
   - Métriques contextuelles

3. **🎯 Analyses Tactiques**
   - Détection automatique de formations
   - Analyse de pressing (PPDA)
   - Positions moyennes des joueurs

4. **🔍 Scouting Intelligent**
   - Profilage automatique des joueurs
   - Recommandations personnalisées
   - Analyse d'opportunités de transfert

5. **📈 Visualisations**
   - Graphiques radar interactifs
   - Heatmaps de comparaison
   - Évolutions temporelles

### 🚀 Prochaines Étapes

- Intégration de données temps réel
- Modèles prédictifs avancés
- Interface web complète
- APIs externes (Football-Data, Transfermarkt)

---

**🏈 Football Analytics Platform** - *Transformez vos données en avantage compétitif*